In [1]:
import pandas as pd 

full_data_info = pd.read_csv('2000零件.csv',encoding ='utf-8')

In [2]:
full_data_info['日期'] = pd.to_datetime(full_data_info['日期'], format='%Y%m%d').dt.strftime('%Y-%m-%d')

In [3]:
# 零件号，仓库，根据20200101-202112月31号订单计算阈值，单条，多条，20230101号拦截订单数量，批次号
# 零件号，仓库，根据20200102-202301月01号订单计算阈值，单条，多条，20230102号拦截订单数量，批次号
# 零件号，仓库，根据20200103-202301月02号订单计算阈值，单条，多条，20230103号拦截订单数量，批次号

# 批次号，订单号，订单数量，零件号，仓库，经销商，上次订单间隔

In [4]:
import numpy as np

def find_outliers_onehot(data):  
    
    if len(data)==1 or len(set(data))==1:
        one_hot_num = float(data.head(1))
        if one_hot_num<=2000:
            thr_sig_num =  one_hot_num*2.5
        else:
            thr_sig_num = one_hot_num*1.5
    else:
        thr_sig_num  = np.nan 
    return thr_sig_num

def find_outliers_3sigma(data):  
    
    if len(data)==1 or len(set(data))==1:
        thr_sig_num = np.nan
    else:
        mean = sum(data) / len(data)  
        std_dev = (sum((x - mean) ** 2 for x in data) / len(data)) ** 0.5  
        thr_sig_num  = mean + 3 * std_dev 
    return thr_sig_num

def find_outliers_xxt(x):  
    
    if len(x)==1 or len(set(x))==1:
        thr_sig_num = np.nan
    else:
        q3 = lambda x: x.quantile(0.75)  
        q1 = lambda x: x.quantile(0.25)  
        thr_sig_num = q3(x) + 1.5 * (q3(x) - q1(x))  
    
    return thr_sig_num

def data_proces_info(df,type_info):
    df.columns = ['零件号','仓库','type',type_info+'_counts']
    df = df[df['type']==0]
    return df.drop(columns= ['type'])

In [5]:
def get_outlier_sys_info(full_data_info,start_time,end_time,test_time):
    
    data_train = full_data_info[(full_data_info['日期']<end_time)&(full_data_info['日期']>=start_time)]
    
    data_test = full_data_info[(full_data_info['日期']<=test_time)&(full_data_info['日期']>=end_time)]

    thr_sigma_info_onehot = data_train.groupby(['零件号' ,'仓库'])['需求数量'].apply(find_outliers_onehot).reset_index()
    thr_sigma_info_onehot = thr_sigma_info_onehot.rename(columns={'需求数量': 'onehot_outlier_values'}) 

    thr_sigma_info_3sigma = data_train.groupby(['零件号' ,'仓库'])['需求数量'].apply(find_outliers_3sigma).reset_index()
    thr_sigma_info_3sigma = thr_sigma_info_3sigma.rename(columns={'需求数量': '3sigma_outlier_values'})  

    thr_sigma_info_xxt = data_train.groupby(['零件号' ,'仓库'])['需求数量'].apply(find_outliers_xxt).reset_index()
    thr_sigma_info_xxt = thr_sigma_info_xxt.rename(columns={'需求数量': 'xxt_outlier_values'})  


    merged_df_onehot_sigma = pd.merge(thr_sigma_info_onehot, thr_sigma_info_3sigma,
                                  on=['零件号' ,'仓库'],how ='left')
    merged_df_full = pd.merge(merged_df_onehot_sigma, thr_sigma_info_xxt,
                                      on=['零件号' ,'仓库'],how ='left')



    data_test_outlier_values = pd.merge(data_test, merged_df_full,
                                      on=['零件号' ,'仓库'],how ='left')
    
    data_test_outlier_onehot = data_test_outlier_values.dropna(subset='onehot_outlier_values')
    data_test_outlier_onehot = data_test_outlier_values.dropna(subset='onehot_outlier_values')
    data_test_outlier_notonehot = data_test_outlier_values.dropna(subset='3sigma_outlier_values')

    data_test_outlier_notonehot['箱型图阈值拦截情况'] = np.where(data_test_outlier_notonehot['需求数量']
                                                     <= data_test_outlier_notonehot['xxt_outlier_values'], 1, 0)
    data_test_outlier_notonehot['3sigma阈值拦截情况'] = np.where(data_test_outlier_notonehot['需求数量']
                                                     <= data_test_outlier_notonehot['3sigma_outlier_values'], 1, 0)
    data_test_outlier_onehot['onehot拦截情况'] = np.where(data_test_outlier_onehot['需求数量']
                                                 <= data_test_outlier_onehot['onehot_outlier_values'], 1, 0) 

    
    
    thr_sigma_sample_info  = data_test_outlier_notonehot[['零件号','仓库','经销商代码','需求数量','日期','3sigma阈值拦截情况']]
    thr_sigma_sample_info = thr_sigma_sample_info[thr_sigma_sample_info['3sigma阈值拦截情况']==0]


    his_sample_info = full_data_info[(full_data_info['日期']<test_time)]
    his_sample_info.sort_values(by=['零件号','仓库','经销商代码','日期'],inplace =True)
    his_sample_info['Last_Order_Time'] = his_sample_info.groupby(['零件号','仓库','经销商代码']).日期.shift(1)  
    his_sample_info  = his_sample_info[['零件号','仓库','经销商代码','需求数量','日期','Last_Order_Time']]
    his_sample_info = his_sample_info[(his_sample_info['日期']<=test_time)&(his_sample_info['日期']>=end_time)].dropna(subset=['Last_Order_Time'])

    thr_sigma_sample_output = pd.merge(thr_sigma_sample_info,his_sample_info,on = ['零件号','仓库','经销商代码','需求数量','日期'],how ='left')
    
    thr_sigma_sample_output['上次订单间隔（天)'] = pd.to_datetime(thr_sigma_sample_output['日期'])- pd.to_datetime(thr_sigma_sample_output['Last_Order_Time'])
    

    
    output_3sigma = data_test_outlier_notonehot.groupby(['零件号','仓库','3sigma阈值拦截情况']).size().reset_index().sort_values(by=['零件号','仓库'])
                                                                       
    output_xxt = data_test_outlier_notonehot.groupby(['零件号','仓库','箱型图阈值拦截情况']).size().reset_index().sort_values(by=['零件号','仓库'])

    output_onehot = data_test_outlier_onehot.groupby(['零件号','仓库','onehot拦截情况']).size().reset_index().sort_values(by=['零件号','仓库'])

    

    output_onehot_new=data_proces_info(output_onehot,'one_hot')
    output_3sigma_new=data_proces_info(output_3sigma,'3sigma')
    output_xxt_new=data_proces_info(output_xxt,'xxt')

    df_list = [output_onehot_new,output_3sigma_new,output_xxt_new]  
  
    # 创建一个空的DataFrame，用作合并后的结果  
  
    # 循环遍历每个DataFrame，将其与结果DataFrame进行合并  
    for df in df_list:  
        merged_df_full = merged_df_full.merge(df, on=['零件号','仓库'],how ='left')
        
    merged_df_full['批次'] = end_time.replace('-','')
    
    thr_sigma_sample_output['批次'] = end_time.replace('-','')

    
    

    merged_df_full.columns = ['零件号', '仓库', '单记录阈值', '3sigma阈值',
       '箱形图阈值', '单记录拦截订单数量', '3sigma拦截订单数量', '箱形图拦截订单数量','批次']
    
    
    
    return merged_df_full,thr_sigma_sample_output
    
    

In [6]:
merged_df_full_01,thr_sigma_sample_output_01 = get_outlier_sys_info(full_data_info,'2022-01-01','2023-01-01','2023-01-31')
merged_df_full_02,thr_sigma_sample_output_02 = get_outlier_sys_info(full_data_info,'2022-02-01','2023-02-01','2023-02-28')
merged_df_full_03,thr_sigma_sample_output_03 = get_outlier_sys_info(full_data_info,'2022-03-01','2023-03-01','2023-03-31')
merged_df_full_04,thr_sigma_sample_output_04 = get_outlier_sys_info(full_data_info,'2022-04-01','2023-04-01','2023-04-30')
merged_df_full_05,thr_sigma_sample_output_05 = get_outlier_sys_info(full_data_info,'2022-05-01','2023-05-01','2023-05-31')
merged_df_full_06,thr_sigma_sample_output_06 = get_outlier_sys_info(full_data_info,'2022-06-01','2023-06-01','2023-06-30')


frames = [merged_df_full_01, merged_df_full_02, merged_df_full_03
         ,merged_df_full_04,merged_df_full_05,merged_df_full_06]  
output = pd.concat(frames) 
output.sort_values(by=['零件号',
                       '仓库','批次'],inplace =True)



output.to_excel('拦截情况统计.xlsx')

sample_frames = [thr_sigma_sample_output_01, thr_sigma_sample_output_02, thr_sigma_sample_output_03
         ,thr_sigma_sample_output_04,thr_sigma_sample_output_05,thr_sigma_sample_output_06]  
sample_output = pd.concat(sample_frames) 
sample_output.sort_values(by=['零件号',
                       '仓库','批次'],inplace =True)
sample_output.to_excel('拦截情况明细.xlsx')


/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_23146/261596736.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test_outlier_notonehot['箱型图阈值拦截情况'] = np.where(data_test_outlier_notonehot['需求数量']
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_23146/261596736.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test_outlier_notonehot['3sigma阈值拦截情况'] = np.where(data_test_outlier_notonehot['需求数量']
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_23146/261596736.py

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_23146/261596736.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  his_sample_info['Last_Order_Time'] = his_sample_info.groupby(['零件号','仓库','经销商代码']).日期.shift(1)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_23146/261596736.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test_outlier_notonehot['箱型图阈值拦截情况'] = np.where(data_test_outlier_notonehot['需求数量']
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_23146/26159673

In [7]:
# full_data_info[(full_data_info['零件号']=='56D945105')&(full_data_info['仓库']=='6000')
#               &(full_data_info['经销商代码']==13572)]

In [8]:
full_data_info.head()

,零件号,日期,需求数量,仓库,vbeln,posnr,经销商代码,需求频次abc分类,价格abc分类,需求数量ABC分类,零件代码第四位,EOP,计划员
0,06F260849E,2019-06-14,3,8023,6589989,900,12538,B,C,MIDDLE,2.0,716.0,4
1,6R0821141B,2019-06-14,5,8023,6590828,900,12429,A,D,FAST,8.0,619.0,3
2,6R0821141B,2019-09-05,1,8023,6935935,14200,10408,A,D,FAST,8.0,619.0,3
3,6R0821141B,2019-09-05,2,8023,6935783,2500,10538,A,D,FAST,8.0,619.0,3
4,6R0821141B,2019-09-05,1,8023,6932590,800,12586,A,D,FAST,8.0,619.0,3


In [9]:
fst_sec_rate_info  = full_data_info.sort_values(['零件号', '日期'])  
fst_sec_rate_info['需求数量'] = fst_sec_rate_info['需求数量'].astype(str)
fst_sec_rate_info = fst_sec_rate_info.groupby(['零件号','仓库'])['需求数量'].agg('|'.join).reset_index() 


fst_sec_rate_info['第一次需求数量'] = [x.split('|')[0] for x in fst_sec_rate_info['需求数量']]
fst_sec_rate_info['第二次需求数量'] = [x.split('|')[1] if len(x.split('|'))>1 else 'nan' for 
                                x in fst_sec_rate_info['需求数量']]
fst_sec_rate_info = fst_sec_rate_info[fst_sec_rate_info['第二次需求数量']!='nan']
for i in ['第一次需求数量','第二次需求数量']:
    fst_sec_rate_info[i] =[float(x) for x in fst_sec_rate_info[i]]
    
fst_sec_rate_info['rate'] = fst_sec_rate_info['第二次需求数量']/fst_sec_rate_info['第一次需求数量']
fst_sec_rate_info.to_excel('单记录第一次第二次需求值比率分布情况.xlsx')

In [10]:
fst_sec_rate_info

,零件号,仓库,需求数量,第一次需求数量,第二次需求数量,rate
0,000098602A,1000-1,1|1|1,1.0,1.0,1.000000
1,000098670A,1000-1,1|1|5|5|1|1|5|1|1|1|1|1|1|1|1|1|1|1|1|1|1,1.0,1.0,1.000000
2,000800981B,1000-2,1|1|1|1,1.0,1.0,1.000000
3,000800981C,1000-2,1|1|2|1,1.0,1.0,1.000000
4,000979035E,1000-1,1|1|1|1|1|1,1.0,1.0,1.000000
...,...,...,...,...,...,...
5794,WHT007210,1100-3,4|4|4|4|4|4,4.0,4.0,1.000000
5795,WHT007803,1000-1,20|50|30,20.0,50.0,2.500000
5796,WHT008715,1000-1,1|6|6|2|4,1.0,6.0,6.000000
5797,WHT008801,1000-1,10|5|2|5|1|10|10|3|3,10.0,5.0,0.500000
